# Tutorial for SMT based Verification

As an example we try to verifiy the robustness of a classification Network trained on the iris dataset. We learn how to train a small neural network in pytorch and use the wheigts and biases to create a number of terms and formulas which can be solved by a SMT solver.


In [ ]:
# imports for torch
import torch
from torch import nn
from torch.autograd import Variable

# imports for sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# import for progress bar
import tqdm

# numpy
import numpy as np

# for plots
import matplotlib.pyplot as plt

# for measuring runtime
import time

# SMT solver
%pip install z3-solver
from z3 import *

## Training of Neural Network

In [ ]:
import torch.nn.functional as F
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, number_of_neurons):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
!wget --no-cache --backups=1 {}

## Creating Formulas for the SMT solver

## Verifying robustness